In [ ]:
import numpy as np
import scipy.io
from itertools import combinations, product
from scipy.special import comb
import pickle as pkl
import os
import matplotlib.pyplot as plt

In [ ]:
def import_data(patient, d=3):
    '''...'''
    data = scipy.io.loadmat(f'/Users/anibal/Google Drive/Information cochains/O-info/OinfoSinfo/OinfoCopulasN{d+1}/Oinfop{patient}.mat')

    del data['__header__']
    del data['__version__']
    del data['__globals__']

    data['TC']  = (data['Sinfo'] + data['Oinfo'])/2
    data['DTC'] = (data['Sinfo'] - data['Oinfo'])/2
    
    return data

## Structural connectivity

$$SC(i,j) = average\ over\ patients\ of\ sc(i,j)$$

In [ ]:
def construct_edge_weights():
    '''...'''
    conn = scipy.io.loadmat(f'/Users/anibal/Google Drive/Information cochains/connectome/sc_norm_20rois.mat')
    SC = conn['sc'].sum(axis=2) / 161

    edge_weights = {}
    for i in range(20):
        for j in range(i+1,20):
            edge_weights[(i+1,j+1)] = SC[i,j]
            
    return edge_weights

In [ ]:
def construct_simplex_weights(d, r=20):
    '''...'''
    edge_weights = construct_edge_weights()

    simplex_weights = {}
    for simplex in tuple(combinations(range(1,r+1), d+1)):
        simplex_weights[simplex] = min(edge_weights[edge] 
                                        for edge in combinations(simplex,2))

    return simplex_weights

In [ ]:
def construct_boundary(d, r=19):
    '''...'''
    domain_basis = tuple(combinations(range(r+1), d+1))
    target_basis = tuple(combinations(range(r+1), d))
    target_basis_ix = {tuple(v): index for index, v in enumerate(target_basis)}
    N = comb(r+1, d+1, exact=True)
    M = comb(r+1, d, exact=True)
    D = scipy.sparse.csr_matrix((M, N), dtype=np.int8)
    for j in range(d+1):
        jth_faces_ix = [
                target_basis_ix[tuple(np.concatenate((l[:j], l[j+1:])))]
                for l in domain_basis
                ]
        D += scipy.sparse.csr_matrix ( 
                        ([(-1)**j]*N, (jth_faces_ix, range(N))), 
                        shape=(M, N), dtype=np.int8
                        )
    return D

def construct_coboundary(d, r=19):
    '''...'''
    return construct_boundary(d+1,r).T

In [ ]:
def construct_weights(d, inverse=False):
    simplex_weights = construct_simplex_weights(d)
    num_rows = comb(20, d+1, exact=True)
    weights = np.array(list(simplex_weights.values()), dtype=np.float)
    
    if inverse:
        weights = np.reciprocal(weights)
    
    weights = np.reshape(weights, (num_rows,))
    
    return scipy.sparse.csr_matrix (( 
                         weights, (range(num_rows), range(num_rows))), 
                         shape=(num_rows, num_rows), dtype=float
                         )

## Laplacian

$L^{up}_i = W_i^{-1}\, B_{i+1}\, W_{i+1}\, B_i^T$

$L^{down}_i = B_{i-1}^T\, W_{i-1}^{-1}\, B_i\, W_i$

$L = L^{up} + L^{down}$

In [ ]:
print(' d=3', comb(20,4),
      '\n d=4', comb(20,5),
      '\n d=5', comb(20,6) )

In [ ]:
def construct_up_laplacian(d, inverted_weights=False):
    '''Lu = WIa Bp Wp BTa'''
    BTa = construct_coboundary(d)
    Wp = construct_weights(d+1, inverse=False^inverted_weights)
    Bp = construct_boundary(d+1)
    WIa = construct_weights(d, inverse=True^inverted_weights)
    Lu = np.multiply(WIa, np.multiply(Bp ,np.multiply(Wp, BTa)))
    
    return Lu

def construct_down_laplacian(d, inverted_weights=False):
    '''Ld = BTm WIm Ba Wa'''
    Wa = construct_weights(d, inverse=False^inverted_weights)
    Ba = construct_boundary(d)
    WIm = construct_weights(d-1, inverse=True^inverted_weights)
    BTm = construct_coboundary(d-1)
    Ld = np.multiply(BTm, np.multiply(WIm, np.multiply(Ba, Wa)))
    
    return Ld

def construct_laplacian(d, inverted_weights=False):
    '''...'''
    Ld = construct_down_laplacian(d, inverted_weights)
    Lu = construct_up_laplacian(d, inverted_weights)

    return Ld + Lu

## Main

In [ ]:
patients = [f'00{i}' for i in range(1,10) ] \
         + [f'0{i}' for i in range(10,100)] \
         + [f'{i}' for i in range(100,165)]

for d in range(2,3):
    # computing matrix of eigenvectors and storing eigenvalues
    os.mkdir(f'{d}eigendata_sc')
    L = construct_laplacian(d)
    eigens = np.linalg.eig(L.todense())
    np.save(f'{d}eigendata_sc/eigenvalues.npy', eigens[0])
    # decomposing each O-information cochain on the eigenbasis
    for patient in patients:    
        data = import_data(patient, d)
        cochain = data['Oinfo']
        x = np.linalg.solve(eigens[1], cochain)
        np.save(f'{d}eigendata_sc/{d}harmonic{patient}.npy', x) #save